In [1]:
import json
import numpy as np
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis
from portfolio_hubo_qaoa_light import HigherOrderPortfolioQAOA
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov

# Load experiments data
experiments = None
with open("experiments_data.json", "r") as f:
    experiments = list(json.load(f)["data"])

example_id = 13

# Load data
experiment = experiments[example_id]

print("Data: ", experiment)

stocks = experiment["stocks"]
start = experiment["start"]
end = experiment["end"]
risk_aversion = 3
max_qubits = 15
budget = experiment["budget"]
print(f"Budget: {budget}")

data = yf.download(stocks, start=start, end=end)
prices_now = data["Close"].iloc[-1]
    
returns = data["Close"].pct_change().dropna()
stocks = returns.columns
print("Stocks: ", stocks)


#print(numpy_returns.shape[0])
expected_returns = returns.add(1).prod() ** (252 / len(returns)) - 1
print("Expected returns: ", expected_returns)
#expected_returns = expected_returns.to_numpy()

expected_returns = mean_historical_return(data["Close"]).to_numpy() #numpy_returns.mean(axis=0) * numpy_returns.shape[0] #* 252 #* 10
covariance_matrix = sample_cov(data["Close"]).to_numpy()
numpy_returns = returns.to_numpy()

print("Returns: ", numpy_returns.mean(axis=0)*252)
coskewness_tensor = coskewness(data["Close"].pct_change(fill_method=None).dropna(how="all").to_numpy())
cokurtosis_tensor = cokurtosis(data["Close"].pct_change(fill_method=None).dropna(how="all").to_numpy())

print("Expected returns: ", expected_returns)
print("Covariance matrix: ", covariance_matrix)
print("Coskewness tensor: ", coskewness_tensor)
print("Cokurtosis tensor: ", cokurtosis_tensor)

print()

# Print example value from each moment
print(f"Expected returns: {expected_returns[0]}")
print(f"Covariance matrix: {covariance_matrix[0][1]}")
print(f"3rd-order coskewness tensor: {coskewness_tensor[0][0][1]}")
print(f"4th-order cokurtosis tensor: {cokurtosis_tensor[0][0][0][1]}")

print()

portfolio_hubo = HigherOrderPortfolioQAOA(stocks=stocks,
                                            prices_now=prices_now,
                                            expected_returns=expected_returns, 
                                            covariance_matrix=covariance_matrix,
                                            budget=budget,
                                            max_qubits=max_qubits,
                                            coskewness_tensor=coskewness_tensor, 
                                            cokurtosis_tensor=cokurtosis_tensor,
                                            log_encoding=True,
                                            risk_aversion=risk_aversion)
    
assets_to_qubits = portfolio_hubo.get_assets_to_qubits()

weights, allocation, value, left_overs = portfolio_hubo.solve_with_continuous_variables()

continuous_variables_solution = {
    "weights": weights,
    "allocation": allocation,
    "value": value,
    "left_overs": left_overs
}

print(portfolio_hubo.get_latex_qaoa_circuit())

Data:  {'max_qubits': 3, 'budget': 722.6699981689453, 'stock_metadata': {'DIS': {'max_int': 1, 'qubits': 1, 'random_int': 0, 'price': 111.3499984741211}, 'TRV': {'max_int': 3, 'qubits': 2, 'random_int': 3, 'price': 240.88999938964844}}, 'stocks': ['DIS', 'TRV'], 'start': '2015-01-01', 'end': '2025-01-01', 'n_stocks': 2}
Budget: 722.6699981689453
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


Stocks:  Index(['DIS', 'TRV'], dtype='object', name='Ticker')
Expected returns:  Ticker
DIS    0.025852
TRV    0.111174
dtype: float64
Returns:  [0.0641842  0.13474382]
Expected returns:  [0.02585245 0.11117374]
Covariance matrix:  [[0.07750484 0.02601209]
 [0.02601209 0.05768082]]
Coskewness tensor:  [[[ 0.32992632 -0.04781468]
  [-0.04781468 -0.529501  ]]

 [[-0.04781468 -0.529501  ]
  [-0.529501   -1.35292153]]]
Cokurtosis tensor:  [[[[11.30963637  1.775125  ]
   [ 1.775125    2.28773511]]

  [[ 1.775125    2.28773511]
   [ 2.28773511  5.14398784]]]


 [[[ 1.775125    2.28773511]
   [ 2.28773511  5.14398784]]

  [[ 2.28773511  5.14398784]
   [ 5.14398784 24.12404197]]]]

Expected returns: 0.02585245265673941
Covariance matrix: 0.0260120942446283
3rd-order coskewness tensor: -0.04781467520226737
4th-order cokurtosis tensor: 1.7751250007153265

Number of qubits for asset DIS: 3
Number of qubits for asset TRV: 2
Total number of qubits:  5
Optimized Weights (considering variance, skewne